In [188]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, ShuffleSplit
import networkx as nx

from common.adjacency_list_to_graph import build_graph
from common.calculate_spring_rank import calculate_spring_rank
from common.graph_to_matrix import build_matrix

import itertools

from pprint import pprint

%matplotlib inline
import matplotlib.pyplot as plt

In [464]:
transactions_df = pd.read_csv("./part_data_big", sep=" ", names=["from", "to", "value", "block"])
transactions_df["value"] = 1
transactions_df = transactions_df.sort_values("block")

In [280]:
transactions_df = pd.read_csv("./twitter_combined.txt", sep=" ", names=["from", "to"])
transactions_df["value"] = 1

In [465]:
def create_dataset(dataset, addresses=None):
    if addresses:
        dataset = dataset[dataset["from"].isin(addresses) & dataset["to"].isin(addresses)]
    return dataset.groupby(["from", "to"])["value"].sum().to_frame()

In [466]:
def find_ranks(dataset, alpha):
    edges = dataset["value"].to_dict()
    graph = build_graph(edges)
    nodes = list(graph)
    A = build_matrix(graph, nodes)
    iterations, raw_rank = calculate_spring_rank(A, alpha=alpha)
    
    rank = pd.DataFrame()
    rank["address"] = nodes
    rank["rank"] = raw_rank
    
    return rank.set_index("address")

Модель предсказания дуг:
$$P_{ij} = \frac{1}{1 + e^{-2\beta(s_i - s_j)}}$$
$$P_{ji} = \frac{1}{1 + e^{2\beta(s_i - s_j)}} $$
$$P_{ij} = 1 - P_{ji}$$

$$E_{ij} = c \exp{-\frac{\beta}{2}(s_i - s_j - 1)^2}$$

In [467]:
# def predict_edges(dataset, ranks, beta):
#     dataset["from_rank"] = ranks.loc[[a for a, _ in dataset.index]]["rank"].tolist()
#     dataset["to_rank"] = ranks.loc[[b for _, b in dataset.index]]["rank"].tolist()
#     return (1 / (1 + np.exp(-2 * beta * (dataset["from_rank"] - dataset["to_rank"])))).tolist()

In [468]:
def predict_edges(dataset, ranks, beta, c):
    dataset["from_rank"] = ranks.loc[[a for a, _ in dataset.index]]["rank"].tolist()
    dataset["to_rank"] = ranks.loc[[b for _, b in dataset.index]]["rank"].tolist()
    dataset["direction_probability"] = (1 / (1 + np.exp(-2 * beta * (dataset["from_rank"] - dataset["to_rank"]))))
    dataset["number_of_edges"] = c * np.exp(- beta / 2 * (dataset["from_rank"] - dataset["to_rank"] - 1) ** 2)
    return dataset["direction_probability"].tolist(), dataset["number_of_edges"].tolist()

Multigraph accuracy:
$$\sigma_a = 1 - \frac{1}{2M}\sum_{i,j}|{A_{ij} - (A_{ij} + A_{ji})P_{ij}}|$$

M - сумма всех весов

Как пропустить ошибку для отсутствующих дуг?
- Если Aij = 0 и Aji = 0 - ошибку можно пропустить?
- Если Aij = 0 и Aji != 0
$$A^*_{ij} = A_{ij}$$
$$e_{ij} = |0 - A_{ji}(1 - P_{ji})| = A_{ji}(1 - P_{ij})$$
$$e_{ij} + e_{ji} = |A_{ji} - A_{ji}P_{ji}| + A_{ji} - A_{ji}P_{ij} = 2(A_{ji} - A_{ji}P_{ij}) $$
Т.е. для непарных дуг удваиваем ошибку

Предполагается, что:
$$A_{ij} = 0, A_{ji} = 0 \rightarrow P_{ij} = 0, P_{ji} = 0$$

Есть ли возможность проверять предсказанное количество дуг?

Есть, если вместо Aij + Aji использовать
$$E_{ij} = c \exp{-\frac{\beta}{2}(s_i - s_j - 1)^2}$$

А вместо 2M?
$$ M + \sum_{ij}E_{ij} $$

$$E_{ji}(1 - P_{ji}) + |A_{ji} - E_{ji}P_{ji}|$$

In [469]:
# def accuracy(dataset, predictions):
#     accuracy_dataset = dataset.copy()
#     accuracy_dataset = accuracy_dataset.merge(accuracy_dataset.reset_index(), left_index=True, right_on=["to", "from"], how="left", suffixes=('', '_reversed'))
#     accuracy_dataset["prediction"] = predictions
#     accuracy_dataset["total_value"] = accuracy_dataset["value"] + accuracy_dataset["value_reversed"].fillna(0)
#     accuracy_dataset["paired"] = accuracy_dataset["value"] < accuracy_dataset["total_value"]
#     accuracy_dataset["error"] = accuracy_dataset["value"] - accuracy_dataset["total_value"] * accuracy_dataset["prediction"]
#     accuracy_dataset.loc[~accuracy_dataset["paired"], "error"] *= 2
#     return 1 - np.abs(accuracy_dataset["error"]).sum() / 2 / accuracy_dataset["value"].sum()

In [470]:
def accuracy(dataset, direction, edges):
    accuracy_dataset = dataset.copy()
    accuracy_dataset = accuracy_dataset.merge(accuracy_dataset.reset_index(), left_index=True, right_on=["to", "from"], how="left", suffixes=('', '_reversed'))
    accuracy_dataset["not_paired"] = 0
    accuracy_dataset.loc[np.isnan(accuracy_dataset["value_reversed"]), "not_paired"] = 1
    accuracy_dataset["direction"] = direction
    accuracy_dataset["edges"] = edges
    accuracy_dataset["prediction"] = accuracy_dataset["direction"] * accuracy_dataset["edges"]
    accuracy_dataset["error"] = np.abs(accuracy_dataset["value"] - accuracy_dataset["prediction"])
    accuracy_dataset["non_paired_error"] = accuracy_dataset["not_paired"] * accuracy_dataset["edges"] * (1 - accuracy_dataset["direction"])
    return 1 - \
        (accuracy_dataset["error"].sum() + accuracy_dataset["non_paired_error"].sum()) / \
        (accuracy_dataset["value"].sum() + accuracy_dataset["edges"].sum()) 

In [471]:
test_df = pd.DataFrame()
test_df["from"] = ["0x1", "0x2", "0x3"]
test_df["to"] = ["0x2", "0x1", "0x4"]
test_df["value"] = [1, 2, 1]
test_df = test_df.set_index(["from", "to"])

In [472]:
accuracy(test_df, [1/3, 2/3, 1], [3, 3, 1])

1.0

In [ ]:
# alphas = [0]
alphas = np.logspace(-2, 2, 5)
cs = [1.25] # np.linspace(0.5, 1.5, 5)
betas = [10] # np.linspace(10, 40, 5)
split = ShuffleSplit(n_splits=5)
# split = TimeSeriesSplit(n_splits=5)

train_metrics = {}
test_metrics = {}

for train_index, test_index in split.split(transactions_df):
    train = create_dataset(transactions_df.loc[train_index])
    print("Train size: {} samples".format(train.shape[0]))
    train_addresses = list([a for a, _ in train.index] + [b for _, b in train.index])
    test = create_dataset(transactions_df.loc[test_index], addresses=train_addresses)
    print("Test size: {} samples".format(test.shape[0]))
    for alpha in alphas:
        ranks = find_ranks(train, alpha=alpha)
        for beta, c in itertools.product(betas, cs):
            train_directions, train_edges = predict_edges(train, ranks, beta=beta, c=c)
            test_directions, test_edges = predict_edges(test, ranks, beta=beta, c=c)
            train_metrics[(alpha, beta, c)] = train_metrics.get((alpha, beta, c), []) + [accuracy(train, train_directions, train_edges)]
            test_metrics[(alpha, beta, c)] = test_metrics.get((alpha, beta, c), []) + [accuracy(test, test_directions, test_edges)]
            print("Train accuracy: ", train_metrics[(alpha, beta, c)][-1])
            print("Test accuracy: ", test_metrics[(alpha, beta, c)][-1])
pprint(train_metrics)
pprint(test_metrics)

Train size: 738702 samples
Test size: 23326 samples
Graph contains 738702 edges for 884375 nodes
Estimated size of A is 12.4 MB RAM
Matrix A takes 12.4 MB RAM
Matrix has 9.44e-07 density
02:40:15.862487 Calculating Anj ....
02:40:23.331900 Calculating Ajn ....
02:40:28.819097 Calculating A_o ....
02:40:28.898110 Calculating B ....
02:40:29.036086 Matrix B takes 41.9 MB RAM
02:40:29.036205 Calculating b ....
02:40:29.104781 Solving Bx=b equation using 'bicgstab' iterative method
Train accuracy:  0.691464916133643
Test accuracy:  0.5360251245825557
Graph contains 738702 edges for 884375 nodes
Estimated size of A is 12.4 MB RAM
Matrix A takes 12.4 MB RAM
Matrix has 9.44e-07 density
02:41:17.278858 Calculating Anj ....
02:41:24.824231 Calculating Ajn ....
02:41:30.287071 Calculating A_o ....
02:41:30.367743 Calculating B ....
02:41:30.502740 Matrix B takes 41.9 MB RAM
02:41:30.502810 Calculating b ....
02:41:30.568800 Solving Bx=b equation using 'bicgstab' iterative method
Train accuracy: 

In [ ]:
train_metrics_df = pd.DataFrame().from_dict(train_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})
test_metrics_df = pd.DataFrame().from_dict(test_metrics).mean().to_frame().reset_index().rename(columns={"level_0": "alpha", "level_1": "beta", 0: "accuracy", "level_2": "c"})

In [ ]:
test_metrics_df["alpha_log"] = np.log(test_metrics_df["alpha"])
train_metrics_df["alpha_log"] = np.log(train_metrics_df["alpha"])

In [ ]:
plt.plot(train_metrics_df.groupby("alpha_log")["accuracy"].mean())
plt.plot(test_metrics_df.groupby("alpha_log")["accuracy"].mean())

In [ ]:
plt.plot(train_metrics_df.groupby("beta")["accuracy"].mean(), label="train")
plt.plot(test_metrics_df.groupby("beta")["accuracy"].mean(), label="test")
plt.legend()

In [ ]:
plt.plot(train_metrics_df.groupby("c")["accuracy"].mean(), label="train")
plt.plot(test_metrics_df.groupby("c")["accuracy"].mean(), label="test")
plt.legend()

In [479]:
train_metrics_df

,alpha,beta,c,accuracy,alpha_log
0,0,10.0,0.50,0.352605,-inf
1,0,10.0,0.75,0.485899,-inf
2,0,10.0,1.00,0.599171,-inf
3,0,10.0,1.25,0.636972,-inf
4,0,10.0,1.50,0.645568,-inf
5,0,17.5,0.50,0.306024,-inf
6,0,17.5,0.75,0.426170,-inf
7,0,17.5,1.00,0.530298,-inf
8,0,17.5,1.25,0.570286,-inf
9,0,17.5,1.50,0.583437,-inf


In [480]:
test_metrics_df

,alpha,beta,c,accuracy,alpha_log
0,0,10.0,0.50,0.298662,-inf
1,0,10.0,0.75,0.416834,-inf
2,0,10.0,1.00,0.519640,-inf
3,0,10.0,1.25,0.536210,-inf
4,0,10.0,1.50,0.533251,-inf
5,0,17.5,0.50,0.270667,-inf
6,0,17.5,0.75,0.380247,-inf
7,0,17.5,1.00,0.476757,-inf
8,0,17.5,1.25,0.496403,-inf
9,0,17.5,1.50,0.495023,-inf


# Ручная проверка

In [418]:
accuracy(train_showtime, train_showtime["direction_probability"].tolist(), train_showtime["number_of_edges"].tolist())

0.49731028361405405

In [420]:
train_showtime = train.copy()

In [427]:
train_showtime

value  \
from                                       to                                                  
0x0000000000000000000000000000000000000000 0x28a3da09a8194819ae199f2e6d9d1304817e28a5      1   
                                           0xc2836188d9a29253e0cbda6571b058c289a0bb32      1   
0x00000f9ec07c056730188b975ead364e253547fa 0xecf07ef0b6c1522d6c9f668a36f00213036681e5      1   
0x0000985c020fdb2ae61bd7b7a7daca9c2ea047e3 0x9997460531a74a034ff314291e6b6adaef6073ff      1   
0x00009fd1e72f1a635a13a8e4177c697886033ba7 0x4c5c4e1437fc4106f50f40c38878681236b9c12d      1   
0x000171391e7679119cd1811548b9c8c25b6657d0 0x9af8bb6fe0bc253071e92c8cefbe41ebe5abb817      1   
0x0003117510b77b71765b6af931c116f327086f90 0x18adc5cfca4bdee8762e3b333f940fb954461929      1   
0x000438f1de357e597b457decc8bf91964451518b 0x9c927028a91a95a4af841d1feab2de9fbee8c4c4      1   
0x0004c0718b46740eb42b69475db2221e9bb85376 0x8e9a0ad56287e689dc80c27a64fd31e3c4c31373      1   
0x0004db3ed0a60659bf24af1517d245dd7ef75c2c 0x5baeac0a0417a05733884852aa068b706967e790      1   
0x00051d5dba85fb5488f2051b314cee770fb31fff 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x0005842198450f7e51cefa4e609765696a59b29b 0xd2fc3c32c0b25fd950059978fc03f3fde6c960bb      1   
0x000610cbcd54b2ac4a9656ec71eabe75b28fe96d 0x51f65ac6f4361c0dfc34f6ba5cd7ff605a1eb6b3      1   
0x00061c585215fd29a13454262dbddafed6518e05 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89 0x6090a6e47849629b7245dfa1ca21d94cd15878ef      1   
0x0006bf729369e195145d7078038fa816ae07262c 0x2bb66c530704653fe5efff9f56c208e35be4a63f      1   
0x000761de31b1c5a929d66f6851423ca6a7468fd5 0x0b95993a39a363d99280ac950f5e4536ab5c5566      2   
0x0007dbd56862a42c87af88ab1f4e32fe72823a8b 0xc715748df02c8f32412bba8f1607ce1bfe740a74      1   
0x00081af78944770f74302d3c685dfdb55ca75ffc 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x0008831df723b3064cfc9cbf6bb11f875adb0f16 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x0008f1cbd42286d09e79df38fce907a6c89968a6 0x12444b6ec62e616ebc8a23e56e61f8f4c6da610c      1   
0x00094eb0d46d0465296ab3994c9c34c634bed57d 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x0009cbf26fa8f20f06c63bffb83f93ae49b832a9 0xfa52274dd61e1643d2205169732f29114bc240b3      1   
0x000a8076834dbc2edca1d7692775b6eeca08ee77 0xba11f50e3b3f6ee0e6bae1df5b936570addce24e      1   
0x000af4918b635503689da522d2aeffb7c9bc0f4e 0x0b95993a39a363d99280ac950f5e4536ab5c5566      3   
0x000af6937ff1fd592d53eb5db2d9eb7855bfcc26 0x0b95993a39a363d99280ac950f5e4536ab5c5566      1   
0x000bf0664e5c3e321cd62d83fe6ecf14651de125 0x1303aa3ead0401b55c140b2d8a7eed7fe338a79e      1   
                                           0xf9c6ff62ad50e93652780decec989c7d1cbd4f0e      1   
0x000c700ad310fac642e873e5c86375d8ca9e040f 0x7415c7bf3e2415fa9a55f1fd8b6fccf2914c39a6      1   
0x000d4b94db7dd666b8a6bc445798b211a042abc8 0x015552a658bfe445a3da01c74f6b01d4c84c1476      1   
...                                                                                      ...   
0xffc81c3ce2d090259126ea41f5553de630790166 0x56eb83949fdf543e8cee4569739d21f07e4eafa9      1   
0xffc9983d978ca34a7fedd569bec403068b7f8200 0xd772b4707e8c7b3c602594adcde4e0a28d97ff3c      1   
0xffcac667b14280a9be3cdeca11942a0210c4ceb9 0x9b163e051b820e651c63944ce5de9002774bae5b      1   
0xffcb304e3d8727ac93e914db8fa2d4214973f3d9 0xb51b7454273118990c7599bdb54055cb2df8ca20      1   
0xffce56001335560e765f66ef45a98cf2b00070bc 0x61c22d6953d0ce270a98ef7737ce3b86e510b758      1   
0xffd21afda11c65e3cccb2bc544e0c9f35a74e7c8 0x92a2ede8b85e592e2e8c0dfc04e14c4ae9480240      1   
0xffd53c6654dedc39184a418c2eccbf2811763707 0xb2913ac117ab9e28c91a3dda479ac271765c4631      1   
0xffd82a90a0d1e2d58d2dfd1c04deb826cba689d0 0x3be1908e493327dcace2dd38c7f83e30813b87fa      1   
0xffda43bb7f5f7ad17215f4d2efb07181127146ac 0xffda43bb7f5f7ad17215f4d2efb07181127146ac      1   
0xffda94636747d3305912873d32ff31e364961409 0xfbb1b73c4f

In [429]:
accuracy(train_showtime, train_showtime["direction_probability"].tolist(), train_showtime["number_of_edges"].tolist())

0.6555236015738831

In [430]:
train_showtime["prediction"] = train_showtime["direction_probability"] * train_showtime["number_of_edges"]

In [462]:
train_showtime["number_of_edges"].max()

1.4999999925556873

In [446]:
((train_showtime["value"] - train_showtime["prediction"]) ** 2).sort_values(ascending=True).head(30000)

from                                        to                                        
0x0fd081e3bb178dc45c0cb23202069dda57064258  0x8c99da965629c43bd278e25e7959a96f70fdb89b    1.197272e-07
0xb500a1429000ae315717390a052371ffbea59b5c  0x3839416bd0095d97be9b354cbfb0f6807d4d609e    1.345664e-05
0x00a183fdbebce39cc1065b14b1015cea3b40b651  0x3839416bd0095d97be9b354cbfb0f6807d4d609e    1.345664e-05
0x9937e86d5af7b782ba77b535ab075c7d54813771  0x048717ea892f23fb0126f00640e2b18072efd9d2    1.577634e-05
0xf73c3c65bde10bf26c2e1763104e609a41702efe  0x9e75f8a3698ba5f924aad561488e052078b23eda    3.683858e-05
0x1c0d39bb7511653bd2ebea5b4d85608a908ed9f7  0x70bc08efb79c65cd1035f20ecaa124107824572a    5.573184e-05
0x46ac3404a54b3eaf8d3ea687a87eac3bbfb1bd40  0x8bfe5ebb128ee82f4ba80f56bb32409cc87bc6fb    6.079338e-05
0x6fe872e753e1835d7f72ad2942d5a52eeb9b3dde  0x8bfe5ebb128ee82f4ba80f56bb32409cc87bc6fb    6.079338e-05
0x1f2dbb3963257cd90a9b834424690b127cb2b2da  0x8bfe5ebb128ee82f4ba80f56bb32409cc87bc6fb   

Как влияет параметр b на предсказания дуг?

In [354]:
train_predictions

[1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.5097618538803377,
 1.4983119847434916,
 1.4788900742767328,
 1.52233873297785,
 1.52233873297785,
 1.52233873297785,
 1.52233873297785,
 1.5153702480820765,
 1.5222236850639679,
 1.513524216530937,
 1.5221649454468187,
 1.513744149605922,
 1.4648640319063257,
 1.513524216530937,
 1.5212319984445883,
 1.5026188467226878,
 1.4976413413571394,
 1.521729545247866,
 1.521729545247866,
 1.521729545247866,
 1.513524216530937,
 1.5223681137424168,
 1.5223681137424168,
 1.5223